# LLM as a Judge
Paper: Judging LLM-as-a-Judge with MT-Bench and Chatbot Arena, https://arxiv.org/pdf/2306.05685

```text
[System]
Please act as an impartial judge and evaluate the quality of the following code snippet provided for the task described below. Your evaluation should take into account factors such as correctness, efficiency, readability, maintainability, adherence to best practices, and appropriate use of language features. Begin your evaluation with a concise explanation summarizing your assessment. Be thorough and objective, highlighting both strengths and weaknesses. After your explanation, assign a rating on a scale from 1 to 10 using the following format: "Rating: [[rating]]".
[Task Description]
{task_description}
[Code Snippet]
{code_snippet}
[End of Code Snippet]
```

"Among the output-based methods, we find that DeepSeek-V2.5 and GPT-4o outperform other
LLMs without further training."  - Can LLMs Replace Human Evaluators? An Empirical Study of
LLM-as-a-Judge in Software Engineering  https://dl.acm.org/doi/pdf/10.1145/3728963

Prompts: https://github.com/BackOnTruck/llm-judge-empirical/blob/main/source/generation/vanilla_prompt.py

2 Conditions: with and without comments (https://arxiv.org/pdf/2505.16222)

temperature 0, average over 3 runs since its closed source

CoT approach like in  https://arxiv.org/pdf/2505.16222

since we dont provide a task description, correctness cannot be evaluated

Üveys prompt without CoT:

"You are a senior software engineer performing code reviews. "
            "Respond with exactly one word:\n"
            "- 'APPROVE' if the code is correct, readable, and maintainable.\n"
            "- 'CHANGES_REQUESTED' otherwise."


In [4]:
import os
import pandas as pd
import sqlite3

db_path = "../../giicg.db"
if not os.path.exists(db_path):
    raise FileNotFoundError(f"Database file does not exist: {db_path}")

conn = sqlite3.connect(db_path)
code = pd.read_sql("SELECT * FROM translated_scratch_prompts_with_code", conn)
code

,code_reply_id,conversation_id,message_id,message_text,gender,user_id,model_reply,code,model_version,programming_language
0,1,6,5,I want to use Dummy Hot encoding to replace th...,Woman (cisgender),16,You can use `OneHotEncoder` from `sklearn.prep...,from sklearn.preprocessing import OneHotEncode...,chatgpt-4o-latest,python
1,2,7,43,whats the best way to encode and compress a ja...,Man (cisgender),25,Use `JSON.stringify()` to serialize the object...,import LZString from 'lz-string';\n\nfunction ...,chatgpt-4o-latest,javascript
2,3,8,47,I have a pandas dataframe like this:\ndata\tpe...,Woman (cisgender),28,You can use `str.split()` along with `str.extr...,import pandas as pd\n\n# Assuming your DataFra...,chatgpt-4o-latest,python
3,4,10,57,"as a NLP and LLM researcher, I am recently dow...",Non-binary,30,Your idea is **highly practical and implementa...,import os\n import requests\n import time\...,chatgpt-4o-latest,python
4,5,12,65,Blender and Python. I have a collection of hun...,Man (cisgender),34,Here is a **Blender Python script** that creat...,import bpy\nimport mathutils\n\n# === CONFIGUR...,chatgpt-4o-latest,python
...,...,...,...,...,...,...,...,...,...,...
709,710,70,1524,can you write a test that tests whether parse ...,Woman (cisgender),90,Here's a test that verifies `parse_obj` works ...,import pytest\nfrom pydantic import Validation...,deepseek-chat,python
710,711,72,1534,can you write me some python code to count the...,Woman (cisgender),11,Here's a simple Python function to count the n...,import json\n\ndef count_objects_in_json(data)...,deepseek-chat,python
711,712,74,1538,I want to merge these two dataframes: \ndf_tra...,Woman (cisgender),28,You can merge the dataframes using a left merg...,merged_df = df_train_claim.merge(df_train_clai...,deepseek-chat,python
712,713,78,1598,"0,tcp,http,SF,215,45076,0,0,0,0,0,1,0,0,0,0,0,...",Woman (cisgender),73,To determine the best k for k-means clustering...,from sklearn.cluster import KMeans\nfrom sklea...,deepseek-chat,python


## Users per gender

In [5]:
users_per_gender = code.groupby('gender')['user_id'].nunique().reset_index(name='num_users')
users_per_gender

,gender,num_users
0,Another gender,1
1,Man (cisgender),10
2,Non-binary,1
3,Woman (cisgender),8


## Set up LLM

In [6]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate


def evaluate_code(code):

    load_dotenv()

    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    openai_model = "gpt-4o"
    llm = ChatOpenAI(temperature=0.0, model=openai_model)

    class OutputFormat(BaseModel):
        reasoning: str = Field(description="A brief description of your reasoning")
        validity_score: str = Field(description="Your validity judgement score on a scale of 1 to 10")
        cleanliness_score: str = Field(description="Your cleanliness judgement score on a scale of 1 to 10")
        readability_score: str = Field(description="Your readability judgement score on a scale of 1 to 10")
        structure_score: str = Field(description="Your structure judgement score on a scale of 1 to 10")
        best_practices_score: str = Field(description="Your best-practices judgement score on a scale of 1 to 10")

    structured_llm = llm.with_structured_output(OutputFormat)

    system_prompt = SystemMessagePromptTemplate.from_template(
        "You are a software engineer that performs a code review."
    )

    user_prompt = HumanMessagePromptTemplate.from_template(
        """
        You are tasked with rating the quality of the provided code.
        ---
        {code}
        ---
        For each aspect below, think step by step and briefly explain your reasoning (one sentence per aspect):

        1. Validity: Does the code compile and run without errors? Does it use undefined functions or variables?
        2. Cleanliness: Is the code free of code smells, like dead code, large classes, temporary fields, duplicated logic, etc.?
        3. Readability: Is the code clear and easy to understand for a typical developer familiar with the language?
        4. Structure: Is the code logically structured, modular, and well-organized?
        5. Best Practices: Does it follow common language-specific conventions, naming, and style guidelines?

        After your reasoning, give a brief summary and assign a score from 1 to 10 for each aspect. The scoring system is as follows:
        1-2: Poor
        3-4: Below average
        5-6: Average
        7-8: Good
        9-10: Excellent

        If there is no code in the input, leave the output blank.

        """,

    input_variables=["code"]
    )

    complete_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])
    print(f"evaluating next code")
    chain_one = (
            {"code": lambda x: x["code"]}
            | complete_prompt
            | structured_llm
            | {"reasoning": lambda x: x.reasoning,
               "validity_score": lambda x: x.validity_score,
               "cleanliness_score": lambda x: x.cleanliness_score,
               "readability_score": lambda x: x.readability_score,
               "structure_score": lambda x: x.structure_score,
               "best_practices_score": lambda x: x.best_practices_score
               }
    )

    response =  chain_one.invoke({"code": code})

    return response["reasoning"], response["validity_score"],response["cleanliness_score"],response["readability_score"],response["structure_score"],response["best_practices_score"]

In [7]:

code[['reasoning', 'validity_score', 'cleanliness_score', 'readability_score', 'structure_score', 'best_practices_score']] = code['code'].apply(lambda x: pd.Series(evaluate_code(x)))

for i in range(3):
    colnames = [
        f"reasoning_{i}",
        f"validity_score_{i}",
        f"cleanliness_score_{i}",
        f"readability_score_{i}",
        f"structure_score_{i}",
        f"best_practices_score_{i}",
    ]
    code[colnames] = code['code'].apply(
        lambda x: pd.Series(evaluate_code(x)),
    )




evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating next code
evaluating ne

In [8]:
# from scipy.stats import mode
#
# score_cols = ['overall_score_0', 'overall_score_1', 'overall_score_2']
#
# for col in score_cols:
#     code[col] = pd.to_numeric(code[col], errors='coerce')
#
# def majority_vote(row):
#     values = [v for v in row if not pd.isnull(v)]
#     if not values:
#         return None
#     return mode(values, keepdims=False).mode
#
# code['majority_llm_score'] = code[score_cols].apply(majority_vote, axis=1)
#
# code = code[code['gender'].isin(['Woman (cisgender)', 'Man (cisgender)'])].reset_index(drop=True)

In [9]:
# import matplotlib.pyplot as plt
# import seaborn as sns
#
# user_avg = (
#     code.groupby(['model_version', 'user_id', 'gender'])['majority_llm_score']
#     .mean()
#     .reset_index()
# )
#
# # 2. Compute mean per model and gender
# plot_df = (
#     user_avg.groupby(['model_version', 'gender'])['majority_llm_score']
#     .mean()
#     .reset_index()
# )
#
# # 3. Plot!
# plt.figure(figsize=(14, 6))
# sns.set_style("whitegrid")
# bar = sns.barplot(
#     data=plot_df,
#     x='model_version',
#     y='majority_llm_score',
#     hue='gender',
#     palette={'Woman (cisgender)':'red', 'Man (cisgender)':'blue'}
# )
#
# plt.xlabel('Model')
# plt.ylabel('Mean LLM Score')
# plt.title('Rated Code Quality by Model and Gender')
# plt.legend(title='Gender')
# plt.tight_layout()
# plt.show()
#
# summary = code.groupby(['model_version', 'gender']).size().unstack(fill_value=0)
# for llm, counts in summary.iterrows():
#     print(f"LLM: {llm}")
#     for gender, count in counts.items():
#         print(f"  {gender}: {count}")
#


In [10]:
code.to_sql("llm_judge_COT_gpt5", conn, if_exists="replace", index=False)

714

## Averaging over al
exclude 1524 because it contains code
exlcude 764 because it asks the model to something that is bad practice